# 多头注意力

在实践中，给定相同的查询、键、值的组合的时候，我们希望模型可以基于相同的注意力机制学习到不同的行为，然后将不同的行为作为知识组合起来，捕获序列中各种范围的依赖关系，因此允许注意力机制组合使用查询、键和值的不同子空间表示可能是有益的。

对此，我们可以学习得到$h$组不同的线性投影来变换查询、键和值，然后这$h$组变换后的查询、键和值将并行地送入注意力汇聚中。最后这$h$个注意力汇聚的输出拼接在一起，并且通过另一个可以学习的线性投影变换，最终产生输出，这种设计被称为多头注意力，这是在17年被提出的，对于$h$个注意力汇聚输出，每一个注意力汇聚都被称为一个头(head)

## 模型
在实现多头注意力之前，我们使用数学语言将这个模型形式化地描述出来。
给定查询键、值分别为:
$$
\begin{split}\begin{aligned}
\mathbf{q} &\in \mathbb{R}^{d_q}\\
\mathbf{k} &\in \mathbb{R}^{d_k}\\
\mathbf{v} &\in \mathbb{R}^{d_v}\\
\end{aligned}\end{split}
$$
每个注意力头$ \mathbf{h}_i (i=1,\dots,h)$的计算方法为：
$$
\mathbf{h}_i = f(\mathbf{W}_i^{(q)}\mathbf{q},\mathbf{W}_i^{(k)}\mathbf{k},\mathbf{W}_i^{(v)}\mathbf{v}) \in \mathbb{R}^{p_v}
$$
其中可学习的参数有：
$$
\begin{split}\begin{aligned}
\mathbf{W}_i^{(q)} &\in \mathbb{R}^{p_q \times d_q}\\
\mathbf{W}_i^{(k)} &\in \mathbb{R}^{p_k \times d_k}\\
\mathbf{W}_i^{(v)} &\in \mathbb{R}^{p_v \times d_v}\\
\end{aligned}\end{split}
$$

此外，多头注意力的输出还需要经过线性变换，所以它也有可以学习的参数
$$ \mathbf{W}_o \in \mathbb{R}^{p_o \times hp_v} $$
$$ \begin{split}\mathbf W_o \begin{bmatrix}\mathbf h_1\\\vdots\\\mathbf h_h\end{bmatrix} \in \mathbb{R}^{p_o}.\end{split}$$

In [1]:
import sys
sys.path.append("../")
import torch as t
import torch.nn as nn
import math
from pltutils import *

在实现过程中，我们使用缩放点积注意力作为每一个注意力头，为了避免计算代价，设置$ p_q=p_k=p_b = p_o /h_o$ 这样就可以实现并行计算，下面的实现中$p_o$是通过num_hiddens实现的

In [2]:
def transpose_qkv(X:torch.Tensor,num_heads:int):
    """
    为了多注意力的并行计算而转换形状
    """
    # 输入X的形状:(batch_size，查询或者“键－值”对的个数，num_hiddens)
    # 输出X的形状:(batch_size，查询或者“键－值”对的个数，num_heads，num_hiddens/num_heads)
    X=X.reshape(X.shape[0],X.shape[1],num_heads,-1)
    # 输出X的形状:(batch_size，num_heads，查询或者“键－值”对的个数,
    # num_hiddens/num_heads)
    X = X.permute(0, 2, 1, 3)
    # 最终输出 (batch_size*num_heads,查询或者“键－值”对的个数,num_hiddens/num_heads)
    return X.reshape(-1,X.shape[2],X.shape[3])


def transpose_output(X:torch.Tensor, num_heads):
    """逆转transpose_qkv函数的操作"""
    X = X.reshape(-1, num_heads, X.shape[1], X.shape[2])
    X = X.permute(0, 2, 1, 3)
    return X.reshape(X.shape[0], X.shape[1], -1)


class MultiHeadAttention(nn.Module):
    def __init__(self,key_size,query_size,value_size,num_hiddens,num_heads,dropout,bias=False,**kwargs):
        super().__init__(**kwargs)
        self.num_heads = num_heads
        self.attention=DotProductAttention(dropout)
        self.W_q =nn.Linear(query_size,num_hiddens,bias=bias)
        self.W_k=nn.Linear(key_size,num_hiddens,bias=bias)
        self.W_v=nn.Linear(value_size,num_hiddens,bias=bias)
        self.W_o =nn.Linear(num_hiddens,num_hiddens,bias=bias)
    

    def forward(self,queries,keys,values,valid_lens):
        # 将这些玩意拆成batch实现并行化
        # queries, keys, values.shape = (batch_size, num_of_q/k/v s,num_hiddens)
        # valied_lens.shape = (batch_size,num_queries)
        queries=transpose_qkv(self.W_q.forward(queries),self.num_heads)
        keys = transpose_qkv(self.W_k.forward(keys),self.num_heads)
        values = transpose_qkv(self.W_v.forward(values),self.num_heads)

        if valid_lens is not None:
            valid_lens = t.repeat_interleave(valid_lens,repeats=self.num_heads,dim=0)
        output = self.attention(queries, keys, values, valid_lens)

        # output_concat的形状:(batch_size，查询的个数，num_hiddens)
        output_concat = transpose_output(output, self.num_heads)
        return self.W_o(output_concat)


In [13]:
num_hiddens, num_heads = 128, 8
attention = MultiHeadAttention(num_hiddens, num_hiddens, num_hiddens,
                               num_hiddens, num_heads, 0.5)
attention.eval()


# 输入实际数据进行测试
querey= t.zeros((2,1,128))
keys =t.zeros((2,5,128))
values = t.ones((2,5,128))

output = attention.forward(querey,keys,values,None)
print(output.shape)

torch.Size([2, 1, 128])
